In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, f1_score

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def kerasModelResNet50(size=100):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(size, size, 3))

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))


    for layer in base_model.layers:
        layer.trainable = False

    return model


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
size = 100


potholeTrainImages = [os.path.join("/content/drive/MyDrive/Dataset/train/Pothole", img)
                      for img in os.listdir("/content/drive/MyDrive/Dataset/train/Pothole")
                      if img.endswith((".jpg", ".jpeg", ".png"))]

train1 = []
for img_path in potholeTrainImages:
    img = cv2.imread(img_path)
    if img is not None:
        img_resized = cv2.resize(img, (size, size))
        train1.append(img_resized)

temp1 = np.asarray(train1)


nonPotholeTrainImages = [os.path.join("/content/drive/MyDrive/Dataset/train/Plain", img)
                         for img in os.listdir("/content/drive/MyDrive/Dataset/train/Plain")
                         if img.endswith((".jpg", ".jpeg", ".png"))]

train2 = []
for img_path in nonPotholeTrainImages:
    img = cv2.imread(img_path)
    if img is not None:
        img_resized = cv2.resize(img, (size, size))
        train2.append(img_resized)

temp2 = np.asarray(train2)


potholeTestImages = [os.path.join("/content/drive/MyDrive/Dataset/test/Pothole", img)
                      for img in os.listdir("/content/drive/MyDrive/Dataset/test/Pothole")
                      if img.endswith((".jpg", ".jpeg", ".png"))]

test1 = []
for img_path in potholeTestImages:
    img = cv2.imread(img_path)
    if img is not None:
        img_resized = cv2.resize(img, (size, size))
        test1.append(img_resized)

temp3 = np.asarray(test1)

nonPotholeTestImages = [os.path.join("/content/drive/MyDrive/Dataset/test/Plain", img)
                        for img in os.listdir("/content/drive/MyDrive/Dataset/test/Plain")
                        if img.endswith((".jpg", ".jpeg", ".png"))]

test2 = []
for img_path in nonPotholeTestImages:
    img = cv2.imread(img_path)
    if img is not None:
        img_resized = cv2.resize(img, (size, size))
        test2.append(img_resized)

temp4 = np.asarray(test2)


In [ ]:
X_train = np.concatenate((temp1, temp2))
X_test = np.concatenate((temp3, temp4))

y_train1 = np.ones(temp1.shape[0], dtype=int)
y_train2 = np.zeros(temp2.shape[0], dtype=int)
y_test1 = np.ones(temp3.shape[0], dtype=int)
y_test2 = np.zeros(temp4.shape[0], dtype=int)

y_train = np.concatenate((y_train1, y_train2))
y_test = np.concatenate((y_test1, y_test2))

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print("train shape X", X_train.shape)
print("train shape y", y_train.shape)


train shape X (700, 100, 100, 3)
train shape y (700, 2)


In [6]:
inputShape = (size, size, 3)
model = kerasModelResNet50()

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1)

metrics = model.evaluate(X_test, y_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

print("Saving model weights and configuration file")
model.save('sample_resnet50.h5')

model_json = model.to_json()
with open("sample_resnet50.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("sample_resnet50.weights.h5")
print("Saved model to disk")


# Predict class probabilities
y_pred_probs = model.predict(X_test)

# Convert probabilities to class labels
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification Report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=["Plain", "Pothole"]))

# F1 Score (Binary)
f1 = f1_score(y_true_classes, y_pred_classes)
print("F1 Score (binary):", f1)

# Optional: Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
print("\nConfusion Matrix:")
print(cm)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


NameError: name 'X_train' is not defined